
<a href="https://colab.research.google.com/github/kokchun/Databehandling-AI22/blob/main/Lectures/L1-missing-aggregation.ipynb" target="_parent"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a> &nbsp; for interacting with the code

---
# Lecture notes - Pandas - missing data & aggregations

---
This is the lecture note for **Pandas missing data & aggregations** - but it's built upon contents from previous course: 
- Python programming

<p class = "alert alert-info" role="alert"><b>Note</b> that this lecture note gives a brief introduction to Pandas. I encourage you to read further about pandas.

Read more 

- [documentation - read_csv](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html)

- [documentation - aggregate](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.aggregate.html)

- [documentation - groupby](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html)

- [documentation - missing values](https://pandas.pydata.org/pandas-docs/stable/user_guide/missing_data.html)
---

## Setup
- create dummy DataFrame
- it's very useful to create dummy DataFrame in order to test and learn Pandas methods

1. create a 6*6 size array that has the number from 1 to 9
2. select 10 index from random flatt matrix
3. make the matrix flatt


In [60]:
import numpy as np
import pandas as pd
import seaborn as sns
np.random.seed(36)#make sure is not more than 64
n=6
random_matrix=np.random.randint(1,10,size=(n,n)).astype(float)

index=np.random.choice(n*n,10,replace=False)
rval=random_matrix.ravel()[index]=np.nan

print("random matrix:\n",random_matrix)
df=pd.DataFrame(random_matrix)
df
#df.columns=['A','B','C','D','E','F']
#df

random matrix:
 [[ 6. nan nan  2.  6.  8.]
 [ 7.  5.  1. nan  7.  7.]
 [ 2. nan nan  5.  4.  4.]
 [ 6.  6.  7.  9.  4. nan]
 [ 6.  6.  1.  8.  4. nan]
 [ 3.  3. nan nan nan  2.]]


,0,1,2,3,4,5
0,6.0,NaN,NaN,2.0,6.0,8.0
1,7.0,5.0,1.0,NaN,7.0,7.0
2,2.0,NaN,NaN,5.0,4.0,4.0
3,6.0,6.0,7.0,9.0,4.0,NaN
4,6.0,6.0,1.0,8.0,4.0,NaN
5,3.0,3.0,NaN,NaN,NaN,2.0


In [ ]:
source=pd.DataFrame(random_matrix,index=[f"player{i}" for i in range(1,n+1)],columns=[f"feature{i}" for i in range(1,n+1)])
source


,feature1,feature2,feature3,feature4,feature5,feature6
player1,6.0,NaN,NaN,2.0,6.0,8.0
player2,7.0,5.0,1.0,NaN,7.0,7.0
player3,2.0,NaN,NaN,5.0,4.0,4.0
player4,6.0,6.0,7.0,9.0,4.0,NaN
player5,6.0,6.0,1.0,8.0,4.0,NaN
player6,3.0,3.0,NaN,NaN,NaN,2.0


---
## Missing data techniques
- None - Pythonic missing data. A Python object - operations done on Python level and not numpy or pandas.
    - Error when performing arithmetic on None dtypes
- NaN - floating point "Not a Number". Supports fast operations, however all arithmetic operations with NaN results in a NaN
- NA - pd.NA -> missing indicator that can be used consistently across data types

Pandas treats NaN and None almost interchangeably
- isnull() - returns true if value is null  
- notnull() - returns true if value is not null
- dropna() - drops the the whole axis with nulls 
- fillna() - fills the null values with certain value

In [67]:
source.notnull()

,feature1,feature2,feature3,feature4,feature5,feature6
player1,True,False,False,True,True,True
player2,True,True,True,False,True,True
player3,True,False,False,True,True,True
player4,True,True,True,True,True,False
player5,True,True,True,True,True,False
player6,True,True,False,False,False,True


In [68]:

source.isnull()

,feature1,feature2,feature3,feature4,feature5,feature6
player1,False,True,True,False,False,False
player2,False,False,False,True,False,False
player3,False,True,True,False,False,False
player4,False,False,False,False,False,True
player5,False,False,False,False,False,True
player6,False,False,True,True,True,False


In [70]:
source.dropna(axis="columns")

,feature1
player1,6.0
player2,7.0
player3,2.0
player4,6.0
player5,6.0
player6,3.0


In [72]:
source.fillna(0)#fill all null with 0

,feature1,feature2,feature3,feature4,feature5,feature6
player1,6.0,0.0,0.0,2.0,6.0,8.0
player2,7.0,5.0,1.0,0.0,7.0,7.0
player3,2.0,0.0,0.0,5.0,4.0,4.0
player4,6.0,6.0,7.0,9.0,4.0,0.0
player5,6.0,6.0,1.0,8.0,4.0,0.0
player6,3.0,3.0,0.0,0.0,0.0,2.0


---
## Missing data - strategy
Note that there are several strategies to deal with missing data and we will come back to this in the future. 
The strategy you should choose may depend on:
- dataset size
    - small dataset - must be more careful with the data
    - large dataset and small amounts of nulls - can probably remove the rows
- can use different ways to impute missing data
- valuable information (depends on the usage of the data)
- percentage of missing values 
- domain knowledge - how to fill the nulls 
    - e.g. the score example above might be plausible to assume 0 score if the value is missing for certain round
    - e.g. missing some value in a category might be filled with mean or median of that category
    - e.g. missing value between two points could be interpolated
- note that missing values can negatively impact:
    - data visualization
    - arithmetic computations
    - machine learning algorithms 

---
## Aggregations and groupby
Summarizing data

aggegations: 
```python
sum(), mean(), median(), min(), max()
```

Don't confuse with the **aggregate()** or **agg()** for short, method which uses one or more operations over specified axis

In [ ]:
import seaborn as sns
titanic=sns.load_dataset('titanic')
print(titanic.head())


   survived  pclass     sex   age  sibsp  parch     fare embarked  class  \
0         0       3    male  22.0      1      0   7.2500        S  Third   
1         1       1  female  38.0      1      0  71.2833        C  First   
2         1       3  female  26.0      0      0   7.9250        S  Third   
3         1       1  female  35.0      1      0  53.1000        S  First   
4         0       3    male  35.0      0      0   8.0500        S  Third   

     who  adult_male deck  embark_town alive  alone  
0    man        True  NaN  Southampton    no  False  
1  woman       False    C    Cherbourg   yes  False  
2  woman       False  NaN  Southampton   yes   True  
3  woman       False    C  Southampton   yes  False  
4    man        True  NaN  Southampton    no   True  


1. first split the data by two group male\female
2. calculate the avrage of age and survived
3. return the new data form which sex is index
4. i is here male\female
5. 

In [ ]:
#mean(),sum() medium(),min(),max()
survivors=titanic.groupby('sex')[['survived','age']].mean()
 
for i in survivors.index:
    print(i, type(i))
    survived_percentage=survivors.loc[i,'survived']*100#
    print(f"percentage survived {i}: {survived_percentage:.1f}%")
    print(f"mean age {i}: {survivors.loc[i,'age']:.1f}")




female <class 'str'>
percentage survived female: 74.2%
mean age female: 27.9
male <class 'str'>
percentage survived male: 18.9%
mean age male: 30.7


### 📊 Questions for Median

- What is the **median age** of all passengers?
- What is the **median fare** paid by survivors?
- What is the **median age** of passengers in 1st class? with integer or name



In [28]:
median_age=titanic['age'].median()
print(f"median age of all passengers: {median_age:.1f}")

median_fare_survivors=titanic.groupby('survived')['fare'].median()
print(f"median fare of survivors: {median_fare_survivors.loc[1]:.1f}")
print(f"median fare of non-survivors: {median_fare_survivors.loc[0]:.1f}")

media_passanger_1st_class=titanic.groupby('pclass')['age'].median()
print(f"median of age in first class: {media_passanger_1st_class.loc[1]:.1f}")
print(f"median age of third class:{media_passanger_1st_class.loc[3]:.1f}")

#or
median_passenger_first_class=titanic.groupby('class',observed=True)['age'].median()
print(f"medianageof first class:{median_passenger_first_class.loc['First']:.1f}")




median age of all passengers: 28.0
median fare of survivors: 26.0
median fare of non-survivors: 10.5
median of age in first class: 37.0
median age of third class:24.0
medianageof first class:37.0



### ➕ Questions for Sum

- What is the **total number of survivors**?
- What is the **total fare collected** from all passengers?
- What is the **sum of siblings/spouses aboard** for female passengers?



In [35]:
total_survivals=titanic['survived'].sum()
print(f"the total survivals of titanic: {total_survivals}")
total_fare=titanic['fare'].sum()
print(f"the total fare of titanic: {total_fare}")

total_sibling_spouse_female=titanic.groupby(['sex'])['sibsp'].sum()
for i in total_sibling_spouse_female.index:
    if i=='female':
        print(f"total sibs of female:{total_sibling_spouse_female.loc['female']}")

the total survivals of titanic: 342
the total fare of titanic: 28693.9493
total sibs of female:218



### 🔽 Questions for Min

- What is the **minimum age** of any passenger?
- What is the **lowest fare** paid on the Titanic?
- What is the **youngest age** among survivors?


In [ ]:
min_age=titanic['age'].min()
print(f"minimum age of all passengers: {min_age:.1f}")# is 0.42 which is around 5 months
low_fare=titanic['fare'].min()
lowest_paid_fare = titanic[titanic['fare'] > 0]['fare'].min()
print(f"Lowest fare actually paid: {lowest_paid_fare:.2f}")
print(f"lowest fare paid by passengers or nothing paid: {low_fare:.1f}")

youngest_passenger=titanic.groupby('survived')['age'].min()
print(f"youngest survivor age: {youngest_passenger.loc[1]:.1f}")


minimum age of all passengers: 0.4
Lowest fare actually paid: 4.01
lowest fare paid by passengers or nothing paid: 0.0
youngest survivor age: 0.4





### 🔼 Questions for Max

- What is the **maximum age** of passengers who survived?
- What is the **highest fare** paid by any passenger?
- What is the **maximum number of parents/children aboard** for any individual?



In [45]:
max_age=titanic['age'].max()
print(f"maximum age of all passengers: {max_age:.1f}")
highest_fare=titanic['fare'].max()
print(f"highest fare paid by passengers: {highest_fare:.1f}")
max_parents=titanic['parch'].max()
print(f"maximum number of parents/children aboard by a passenger: {max_parents}")

maximum age of all passengers: 80.0
highest fare paid by passengers: 512.3
maximum number of parents/children aboard by a passenger: 6


---

Kokchun Giang

[LinkedIn][linkedIn_kokchun]

[GitHub portfolio][github_portfolio]

[linkedIn_kokchun]: https://www.linkedin.com/in/kokchungiang/
[github_portfolio]: https://github.com/kokchun/Portfolio-Kokchun-Giang

---